In [25]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.interpolate import interp1d
import random
from matplotlib import cm

In [26]:
## Frå githubben


def computeIntegral1D(a,b,func,type=0):
    """_summary_

    Args:
        a (float): start of integraion intervall
        b (float): end of integraion intervall
        func (function): function we want to integrate
        type (int, optional): what integral method we will use

    Returns:
        approx(float) : value of integral
    """
    h=b-a
    if type==0:
        approx = h/2 * (func(-h/(2*np.sqrt(3)) + (a+b)/2,a,b) + func(h/(2*np.sqrt(3)) +(a+b)/2,a,b))
    else: 
        approx = h/2 * (1/3*func(-h/2 + (a+b)/2,a,b) + 4/3*func((a+b)/2,a,b) + 1/3*func(h/2+(a+b)/2,a,b))
    return approx


def compute_order_of_convergence(errors, grid_points):
    orders = []
    for i in range(1, len(errors)):
        # Calculate the ratio of errors and grid sizes
        ratio_errors = np.log(errors[i]) - np.log(errors[i-1])
        ratio_grid = np.log(grid_points[i]) - np.log(grid_points[i-1])
        order = ratio_errors / ratio_grid
        orders.append(order)
    
    # Calculate the average order of convergence
    average_order = np.mean(orders)
    return orders, average_order

### 2   Two dimensional problem

In this section we examine the 2D problem 

\begin{equation*}
    \begin{cases}
        -\Delta u(x,y) = f(x,y), \quad (x,y) \in  (0,1)^2\\
        \nabla u \cdot \nu = g(x,y), \qquad\quad  x,y \in [0,1]  
    \end{cases}
\end{equation*}


#### **2.1**

**Task 1**

We want to find the weak formulation of the problem by finding the functions $a(\cdot, \cdot)$, $F(\cdot)$ and functional space $V$ such that 

$$a(u,v) = F(v), \quad \forall v \in V$$

Denoting the domain as $\Omega = (0,1)^2$ start by integrating the PDE itself using the following identity where $\omega: \Omega \rightarrow \mathbb{R},$ and $ K:\Omega \rightarrow \mathbb{R}^2$

$$
\nabla \cdot (\omega K) = \langle \nabla \omega, K \rangle +  \omega \nabla \cdot K
$$

Setting $\omega = v$ and $K = \nabla u$ and rearranging we get

$$
v \Delta u = v \nabla \cdot \nabla u = -\langle \nabla v, \nabla u \rangle + \nabla \cdot (v\nabla u)
$$

Finally applying this and the divergence theorem in the integrated PDE we get

$$
\int_\Omega f \mathrm{d}x\mathrm{d}y = -\int_\Omega \Delta u \mathrm{d}x\mathrm{d}y = \int_\Omega \langle \nabla v, \nabla u \rangle \mathrm{d}x\mathrm{d}y - \int_\Omega \nabla \cdot (v\nabla u) \mathrm{d}x\mathrm{d}y
$$

$$
= \int_\Omega \langle \nabla v, \nabla u \rangle \mathrm{d}x\mathrm{d}y - \int_{\partial\Omega} v(\nabla u \cdot \nu) \mathrm{d}s = \int_\Omega \langle \nabla v, \nabla u \rangle \mathrm{d}x\mathrm{d}y - \int_{\partial\Omega} vg \mathrm{d}s
$$

Where we used the Neumann condition to get the last integral.

From this we can then define 
$$a(u,v) :=\int_\Omega \langle \nabla v, \nabla u \rangle \mathrm{d}x\mathrm{d}y $$ 
$$F(v) :=\int_\Omega f \mathrm{d}x\mathrm{d}y + \int_{\partial\Omega} vg \mathrm{d}s $$

and since we need first derivatives and there are no special restrictions on what $v$ can be on the boundary we end up with $V = H^1(\Omega)$

**Task 2**

To find the Ritz-Galerkin formulation we write our solution as a linear combinations of basis functions from a finite dimentional subspace of $H^1(\Omega)$

We donte this subspace as $$V_h(\mathcal{T}_h) = \mathrm{span}\{\Psi_i: [0,1] \rightarrow \mathbb{R}: i = 0,...,N\},$$

where $\{\Psi_i\}$ are our basis functions and $\mathcal{T}_h$ is our 2-dimensional mesh. Denoting this approximate solution as $u_h$ we thus get

$$
u_h = \sum_{i=0}^N u_i \Psi_i
$$

We thus get the Ritz-Galerkin formulation as 

$$
a(u_h,\Psi_j) = a\left(\sum_{i=0}^N u_i \Psi_i,\Psi_j\right) = \sum_{i=0}^N u_i a(\Psi_i,\Psi_j) = F(\Psi_j) 
$$
$$\Downarrow$$
$$
\sum_{i=0}^N u_i a(\Psi_i,\Psi_j) = F(\Psi_j), \quad \forall j = 0,...,N
$$

This can then be written as the following linear system given that we define $A_{ij} = a(\Psi_i,\Psi_j)$, $F_j = F(\Psi_j)$ and $U_i = u_i$ 

$$AU = F$$

#### **2.2**

We now want to our basis functions to be the product of one dimensional basis functions. We first define $\mathcal{S}_{h,x}$ and $\mathcal{S}_{h,y}$ to be the set of x and y coordinates of our grid respectively. We then get that our mesh is $ \mathcal{T}_h = \mathcal{S}_{h,x} \times \mathcal{S}_{h,y}.$ 

We then define the 1D basis functions as

$$
V_h(\mathcal{S}_{h,x}) = \mathrm{span} \{\varphi_i: [0,1] \rightarrow \mathbb{R}: i = 0,...,n \} \quad \mathrm{and} \quad V_h(\mathcal{S}_{h,y}) = \mathrm{span} \{\phi_j: [0,1] \rightarrow \mathbb{R}: j = 0,...,n \}
$$

where for every $x_j \in \mathcal{S_{h,x}}$, $y_j \in V_h(\mathcal{S}_{h,y})$

$$
\varphi_i(x_i) = \phi_i (y_j) = \delta_{ij} =

\begin{cases}
    1, \quad i = j \\
    0, \quad i \ne j
\end{cases}
$$


From this we get our 2D basis function as follows

$$
V_h(\mathcal{T}_h) = \mathrm{span} \{\Psi_{ij}: [0,1]^2 \rightarrow \mathbb{R}: \Psi_{ij}(x,y) = \varphi_i (x) \phi_j (y), \varphi_i \in V_h(\mathcal{S}_{h,x}), \phi_j \in V_h(\mathcal{S}_{h,y})  \}
$$

we thus get the following approximation for the solution $u$

$$
u_h(x,y) = \sum_{i,j = 0}^n u_{ij}\Psi_{ij}
$$


**Task 1**

We now choose the labelling function $\ell(i,j) = (n+1)\cdot j + i$, to obtain our ordering of the gridpoints and basis functions.

We now have the that the stiffness matrix $A \in \mathbb{R}^{(n+1)^2\times (n+1)^2} $ is given by

$$
(A)_{\ell(i,j),\ell(l,m)} = \int_\Omega \langle \nabla\Psi_{ij}, \nabla\Psi_{lm} \rangle \mathrm{d}x\mathrm{d}y
$$

to expand this we note that 
$$\nabla \Psi_{ij} = \begin{bmatrix} \partial_x \Psi_{ij} \\ \partial_y \Psi_{ij}  \end{bmatrix}  = \begin{bmatrix} \partial_x (\varphi_i(x)\phi_j(y)) \\ \partial_y (\varphi_i(x)\phi_j(y))  \end{bmatrix} = 
\begin{bmatrix}  \varphi_i'(x)\phi_j(y) \\ \varphi_i(x)\phi_j'(y)  \end{bmatrix} $$

we also define the 1D stiffness matrices $A_x,A_y \in \mathbb{R}^{(n+1)\times(n+1)} $ and 1D mass matrices $M_x, M_y \in\mathbb{R}^{(n+1)\times(n+1)} $ as follows

$$
(A_x)_{il} = \int_0^1 \varphi_i'(x) \varphi_l'(x) \mathrm{d}x, \quad (A_y)_{jm} = \int_0^1 \phi_j'(y) \phi_m'(y) \mathrm{d}y \\
(M_x)_{il} = \int_0^1 \varphi_i(x) \varphi_l(x) \mathrm{d}x,   \quad (M_y)_{jm} = \int_0^1 \phi_j(y) \phi_m(y) \mathrm{d}y
$$

we thus get
$$
(A)_{\ell(i,j),\ell(l,m)} = \int_\Omega [ \varphi_i'(x)\phi_j(y)\varphi_l'(x)\phi_m(y) + \varphi_i(x)\phi_j'(y)\varphi_l(x)\phi_m'(y)] \mathrm{d}x\mathrm{d}y 
= \int_0^1 \varphi_i'(x) \varphi_l'(x) \mathrm{d}x \cdot \int_0^1 \phi_j(y) \phi_m(y)\mathrm{d}y + \int_0^1 \varphi_i(x) \varphi_l(x) \mathrm{d}x \cdot \int_0^1 \phi_j'(y) \phi_m'(y)\mathrm{d}y \\
= (A_x)_{il} \cdot (M_y)_{jm} + (M_x)_{il} \cdot (A_y)_{jm} = (A_y)_{jm} \cdot (M_x)_{il} + (M_y)_{jm} \cdot (A_x)_{il}

$$

we now look at $A_y \otimes M_x + M_y \otimes A_x$ where $\otimes$ denotes the Kronecker product. Comparing our labelling function with the dimensions of the 1D matrices and the definition of the Kronecker product we see that

$$
(A_y \otimes M_x + M_y \otimes A_x)_{\ell(i,j),\ell(l,m)} = (A_y \otimes M_x)_{\ell(i,j),\ell(l,m)} + (M_y \otimes A_x)_{\ell(i,j),\ell(l,m)} = (A_y)_{jm} \cdot (M_x)_{il} + (M_y)_{jm} \cdot (A_x)_{il}
$$

Thus we get that $A = A_y \otimes M_x + M_y \otimes A_x$

In [27]:
class Phi:
    def __init__(self,index):
        self.index = index
    
    def func(self,x,a,b):
        h = b-a
        if self.index == 0:
            return (x-a)/h
        if self.index == 1:
            return (b-x)/h

class Dphi:
    def __init__(self,index):
        self.index = index
    
    def func(self,x,a,b):
        h = b-a
        if self.index == 0:
            return 1/h
        if self.index == 1:
            return -1/h
        
class Integrand2D:
    def __init__(self,index1,index2,type = "O"):
        self.index1 = index1
        self.index2 = index2
        self.type = type         # "O": phi, "D": dphi
    
    def func(self,x,a,b):
        if self.type == "O":
            return Phi(self.index1).func(x,a,b)*Phi(self.index2).func(x,a,b)
        if self.type == "D":
            return Dphi(self.index1).func(x,a,b)*Dphi(self.index2).func(x,a,b)

    

In [ ]:
def massMatrix(mesh, s):
    '''
    Computes the stiffness matrix

    
    '''
    n = len(mesh)
    M = sp.sparse.lil_matrix((n,n))
    for i in range(1,n):            # Sjekk om dette stemmer med oppgåva i 1D
        I1 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,0).func, s)
        I2 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,1).func, s)
        I3 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(1,1).func, s)

        M[i-1,i-1] += I1
        M[i-1,i] += I2
        M[i,i-1] += I2
        M[i,i] += I3
    
    return M.tocsr()



In [ ]:
def stiffnessMatrix(mesh, s):
    '''
    Computes the stiffness matrix

    
    '''
    n = len(mesh)
    A = sp.sparse.lil_matrix((n,n))
    for i in range(1,n):            # Sjekk om dette stemmer med oppgåva i 1D
        I1 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,0,"D").func, s)
        I2 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(0,1,"D").func, s)
        I3 = computeIntegral1D(mesh[i-1], mesh[i], Integrand2D(1,1,"D").func, s)

        A[i-1,i-1] += I1
        A[i-1,i] += I2
        A[i,i-1] += I2
        A[i,i] += I3
    
    return A.tocsr()


In [30]:
def l(n,i,j):
    return (n+1)*j+i

In [31]:
def loadVector2D(mesh_x, mesh_y, s, f):
    """
    
    
    """
    n = len(mesh_x)-1
    F = np.zeros((n+1)**2)


    for j in range(0,n+1):
        for i in range(0,n+1):
            def G(y):

                def integr1(x,a,b):
                    return f(x,y)*Phi(0).func(x,a,b)
                def integr2(x,a,b):
                    return f(x,y)*Phi(1).func(x,a,b)
                


                if i == 0:
                    I1 = 0
                    I2 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr2,s)
                elif i == n:
                    I1 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr1,s)
                    I2 = 0
                else:
                    I1 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr1,s)
                    I2 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr2,s)
                return I1 + I2


            
            def integr3(y,a,b):
                return G(y)*Phi(0).func(y,a,b)
            def integr4(y,a,b):
                return G(y)*Phi(1).func(y,a,b)
            

            if j == 0:
                I3 = 0
                I4 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr4,s)
            elif j == n:
                I3 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr3,s)
                I4 = 0
            else:
                I3 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr3,s)
                I4 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr4,s)


            F[l(n,i,j)] = I3 + I4
    
    return F
        

In [33]:
def applyNeumann2D(mesh_x,mesh_y,s,A,F,g):
    """
    Applies the Neumann conditions to F

    """
    F_new = F.copy()
    n = int(np.sqrt(len(F))) - 1

    for j in set((0,n)):
        for i in range(n+1):
            if j == 0:
                def integr0(x,a,b):
                    return g(x,0)*Phi(0).func(x,a,b)
                def integr1(x,a,b):
                    return g(x,0)*Phi(1).func(x,a,b)
                
                
                if i == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                elif i == n:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1

            if j == n:
                def integr0(x,a,b):
                    return g(x,1)*Phi(0).func(x,a,b)
                def integr1(x,a,b):
                    return g(x,1)*Phi(1).func(x,a,b)
                
                if i == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                elif i == n:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_x[i-1],mesh_x[i],integr0,s)
                    I1 = computeIntegral1D(mesh_x[i],mesh_x[i+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1           

    for i in set((0,n)):
        for j in range(n+1):
            if i == 0:
                def integr0(y,a,b):
                    return g(0,y)*Phi(0).func(y,a,b)
                def integr1(y,a,b):
                    return g(0,y)*Phi(1).func(y,a,b)
                
                if j == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                elif j == n:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1          
            
            if i == n:
                def integr0(y,a,b):
                    return g(1,y)*Phi(0).func(y,a,b)
                def integr1(y,a,b):
                    return g(1,y)*Phi(1).func(y,a,b)
                
                if j == 0:
                    I0 = 0
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                elif j == n:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = 0
                else:
                    I0 = computeIntegral1D(mesh_y[j-1],mesh_y[j],integr0,s)
                    I1 = computeIntegral1D(mesh_y[j],mesh_y[j+1],integr1,s)
                
                F_new[l(n,i,j)] += I0 + I1




    return A, F_new
                



Compiling the method into a single function

In [34]:
def FEMsolve2D(f,g,mesh_x,mesh_y,s=0):
    """
    Solves the finite element method for the 2D problem
    
    """

    M_x = massMatrix(mesh_x,s)
    M_y = massMatrix(mesh_y,s)
    A_x = stiffnessMatrix(mesh_x,s)
    A_y = stiffnessMatrix(mesh_y,s)

    A = sp.sparse.kron(A_y,M_x) + sp.sparse.kron(M_y, A_x)
    A = A.tolil()        # For 
    A[0,0] +=1           # Uniqueness: setting u_00 = 0
    A = A.tocsr()

    F = loadVector2D(mesh_x,mesh_y,s,f)

    A,F = applyNeumann2D(mesh_x,mesh_y,s,A,F,g)

    U = sp.sparse.linalg.spsolve(A,F)

    return U


#### 4, Task 2

We want to compute the error as given by the expression

$$err = \sqrt{\int_{[0,1]^2} ( u_h (x,y)-\bar{u}(x,y))^2} dx dy  = \sqrt{\sum_{i,j=1}^{n} \int_{x_{i-1}}^{x_i} \int_{y_{j-1}}^{y_j} ( u_h (x,y)-\bar{u}(x,y))^2 } dy dx$$

$u_h$ is by definition given by 

$$u_h(x,y) := \sum_{i=0}^{n} \sum_{j=0}^{n} U_{i,j} \varphi_i (x) \phi_j (y) $$

where $U_{i,j} := u_h(x_i,y_j)$. We see that for an element $ [x_{i-1},x_i] \times [y_{j-1},y_j] $ this reduces to 

$$u_h(x,y) = U_{i-1,j-1} \varphi_{i-1} (x) \phi_{j-1} (y) + U_{i,j-1} \varphi_{i} (x) \phi_{j-1} (y) + U_{i-1,j} \varphi_{i-1} (x) \phi_{j} (y) + U_{i,j} \varphi_{i} (x) \phi_{j} (y)$$

In [37]:
def computeError2D(mesh_x,mesh_y,s,u_exact,f,g):
    """
    Compute error in between the exact- and finite element solutions
    """
    n = len(mesh_x)-1


    U = FEMsolve2D(f,g,mesh_x,mesh_y,s).reshape(n+1,n+1)

    err = 0
    for i in range(1,n+1):
        for j in range(1,n+1):
            def G(x,a_x,b_x):

                def integrand(y,a_y,b_y):

                    I11 = U[i-1,j-1] *Phi(1).func(x,a_x,b_x)*Phi(1).func(y,a_y,b_y)
                    I01 = U[i,j-1]   *Phi(0).func(x,a_x,b_x)*Phi(1).func(y,a_y,b_y)
                    I10 = U[i-1,j]   *Phi(1).func(x,a_x,b_x)*Phi(0).func(y,a_y,b_y)
                    I00 = U[i,j]     *Phi(0).func(x,a_x,b_x)*Phi(0).func(y,a_y,b_y)

                    return (I11 + I01 + I10 + I00 - u_exact(x,y))**2
                
                return computeIntegral1D(mesh_y[j-1],mesh_y[j],integrand,s)
            

            err += computeIntegral1D(mesh_x[i-1],mesh_x[i],G)
    

    return np.sqrt(err)
                




In [40]:


def u(x,y):
    pi = np.pi
    return np.sin(2*pi*x)*np.sin(2*pi*y)
    # return np.cos(2*pi*x)*np.cos(2*pi*y)


def f(x,y):
    return 8*np.pi**2*u(x,y)

def g(x,y):
    pi = np.pi
    if x == 0:
        return -2*pi*np.sin(2*pi*y)
    if x == 1:
        return 2*pi*np.sin(2*pi*y)
    if y == 0:
        return -2*pi*np.sin(2*pi*x)
    if y == 1:
        return 2*pi*np.sin(2*pi*x)

    return 0


n_list = [2**i for i in range(2,9+1)]
s = 0

err = np.zeros(len(n_list))

for i,n in enumerate(n_list):
    meshx = np.linspace(0,1,n+1)
    meshy = np.linspace(0,1,n+1)

    X,Y = np.meshgrid(meshx,meshy)
    err[i] = computeError2D(meshx,meshy,s,u,f,g)



In [ ]:
plt.figure()
plt.loglog(n_list,err)



compute_order_of_convergence(err,n_list)[1]
